In [5]:
import numpy as np

In [102]:
table_str = r'''
Baseline &  .79 &  .464 &  .585 &  .643 &  .378 &  .476 & \textbf{.814} &  .297 &  .435 & \textbf{.814} &  .297 &  .435 \\
\midrule
Base & \\
Base \Plus CE &  .685 &  .605 &  .642 &  .535 &  .473 &  .502 &  .588 & \underline{.568} & \underline{.578} &  .588 & \underline{.568} & \underline{.578} \\
Base \Plus CE \Plus FT &  .691 &  .554 &  .615 &  .528 &  .424 &  .47 &  .534 & \textbf{.602} &  .566 &  .534 & \textbf{.602} &  .566 \\
Base \Plus CE \Plus BERT &  .77 &  .572 &  .657 &  .565 &  .42 &  .482 &  .0 &  .0 &  .0 &  .0 &  .0 &  .0 \\
Base \Plus CE \Plus BERT \Minus newly &  .762 &  .618 &  .682 &  .502 &  .407 &  .449 &  .0 &  .0 &  .0 &  .0 &  .0 &  .0 \\
Base \Plus CE \Plus FT \Plus BERT &  .784 & \underline{.724} & \underline{.753} &  .639 & \underline{.589} & \underline{.613} &  .598 &  .542 &  .569 &  .598 &  .542 &  .569 \\
Base \Plus CE \Plus FT \Plus BERT \Minus newly & \underline{.824} &  .644 &  .723 & \underline{.676} &  .528 &  .593 & \underline{.661} &  .538 & \textbf{.593} & \underline{.656} &  .534 & \textbf{.589} \\
Base \Plus CE \Plus FT \Plus BERT (single) & \textbf{.827} & \textbf{.731} & \textbf{.776} & \textbf{.708} & \textbf{.625} & \textbf{.664} &  .492 &  .53 &  .51 &  .472 &  .508 &  .49 \\
\midrule
\multicolumn{13}{l}{ \Plus Fine-tuning (unfreezing) BERT} \\
\midrule
Base \Plus CE \Plus BERT &  .696 &  .61 &  .65 &  .413 &  .362 &  .386 &  .0 &  .0 &  .0 &  .0 &  .0 &  .0 \\
Base \Plus CE \Plus BERT \Minus newly &  .752 &  .718 &  .734 &  .56 &  .534 &  .547 &  .0 &  .0 &  .0 &  .0 &  .0 &  .0 \\
Base \Plus CE \Plus FT \Plus BERT &  .738 &  .678 &  .707 &  .575 &  .528 &  .551 &  .562 &  .5 &  .529 &  .543 &  .483 &  .511 \\
Base \Plus CE \Plus FT \Plus BERT \Minus newly &  .802 &  .689 &  .741 &  .658 &  .565 &  .608 &  .621 &  .521 &  .567 &  .616 &  .517 &  .562 \\
\bottomrule
'''

In [103]:
table_rows = []
string_rows = {}

for i, table_row in enumerate(table_str.split('\n')):
    if table_row == '':
        continue
    
    if table_row.count('&') < 5:
        string_rows[i] = table_row
        continue
        
    table_rows.append(table_row)


In [104]:
def str_is_float(val):
    try:
        return float(val)
    except ValueError:
        return None

def clean_value(val):
    val = val.strip()
    
    if '\\textbf' in val or '\\underline' or r'\\' in val:
        val = val.replace('\\textbf{', '').replace('\\underline{', '').replace('}', '').replace(r'\\', '')
        
       
    float_val = str_is_float(val)
    if float_val is not None:
        return float_val
    else:
        return val
        
table_values = np.array([[clean_value(val) for val in table_row.split('&')[1:]] for table_row in table_rows if table_row.count('&') > 0])
table_rows_headers = [table_row.split('&')[0].strip() for table_row in table_rows]

In [105]:
max_values = []
second_values = []

for i in range(len(table_values[0])):
    current_items = table_values[:, i]
    max_index = np.argmax(current_items)
    max_value = np.max(current_items)
    max_values.append(max_value)
    
    mask = np.ones(current_items.shape, bool)
    mask[max_index] = False
    second_values.append(np.max(current_items[mask]))

In [113]:
def add_string_rows(current_result):
    while True:
        current_entries = current_result.count('\n') + 1
        if current_entries in string_rows.keys():
            current_result += f'{string_rows[current_entries]}\n'
        else:
            break
            
    return current_result

In [114]:
result = ''
for (table_row_header, row_values) in zip(table_rows_headers, table_values):
    result = add_string_rows(result)
    result += f'{table_row_header} & '
    
    for i, row_value in enumerate(row_values):
        if row_value == max_values[i]:
            result += r'\textbf{' + str(row_value)[1:] + '} & '
        elif row_value == second_values[i]:
            result += r'\underline{' + str(row_value)[1:] + '} & '
        else:
            result += f'{str(row_value)[1:]} & '
            
    result = result[:-2]
    result += r'\\' + '\n'
    
result = add_string_rows(result)
print(result)

Baseline & .79 & .464 & .585 & .643 & .378 & .476 & \textbf{.814} & .297 & .435 & \textbf{.814} & .297 & .435 \\
\midrule
Base & \\
Base \Plus CE & .685 & .605 & .642 & .535 & .473 & .502 & .588 & \underline{.568} & \underline{.578} & .588 & \underline{.568} & \underline{.578} \\
Base \Plus CE \Plus FT & .691 & .554 & .615 & .528 & .424 & .47 & .534 & \textbf{.602} & .566 & .534 & \textbf{.602} & .566 \\
Base \Plus CE \Plus BERT & .77 & .572 & .657 & .565 & .42 & .482 & .0 & .0 & .0 & .0 & .0 & .0 \\
Base \Plus CE \Plus BERT \Minus newly & .762 & .618 & .682 & .502 & .407 & .449 & .0 & .0 & .0 & .0 & .0 & .0 \\
Base \Plus CE \Plus FT \Plus BERT & .784 & \underline{.724} & \underline{.753} & .639 & \underline{.589} & \underline{.613} & .598 & .542 & .569 & .598 & .542 & .569 \\
Base \Plus CE \Plus FT \Plus BERT \Minus newly & \underline{.824} & .644 & .723 & \underline{.676} & .528 & .593 & \underline{.661} & .538 & \textbf{.593} & \underline{.656} & .534 & \textbf{.589} \\
Base \Plus C